# 상관분석

* internal_metrics() 이상치를 한번 파악하려고 봤더니 상자그림이 이상하게 그려졌다. 25
* knob 이상치 제거할 필요 없다. 랜덤생성했기 때문에 14 (+ 범주형은 10개)

1. 데이터 전처리
2. 상관분석 함수
3. activedefrag 있는 경우
4. 전체 knob
5. activedefrag 없는 경우
6. pointbeir~


len = 17
cluster0
hz
lru_clock
used_memory_peak
used_memory_overhead
allocator_allocated
allocator_active
maxmemory
maxmemory_human
allocator_frag_bytes
rss_overhead_ratio
rss_overhead_bytes
rdb_last_save_time
instantaneous_ops_per_sec
instantaneous_input_kbps
instantaneous_output_kbps
used_cpu_sys
used_cpu_user

len = 7
cluster1
used_memory_rss
used_memory_rss_human
allocator_resident
allocator_rss_ratio
allocator_rss_bytes
mem_fragmentation_ratio
mem_fragmentation_bytes

In [57]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [58]:
# internal metrics 분류 결과
cluster0 =[ 
'uptime_in_seconds',
'hz',
'lru_clock',
'used_memory_peak',
'used_memory_overhead',
'allocator_allocated',
'allocator_active',
'allocator_resident',
'maxmemory',
'maxmemory_human',
'allocator_frag_bytes',
'allocator_rss_ratio',
'allocator_rss_bytes',
'rdb_last_save_time',
'instantaneous_ops_per_sec',
'instantaneous_input_kbps',
'instantaneous_output_kbps',
'used_cpu_sys',
'used_cpu_user',
]


cluster1 = [
'used_memory_rss',
'used_memory_rss_human',
'rss_overhead_ratio',
'rss_overhead_bytes',
'mem_fragmentation_ratio',
'mem_fragmentation_bytes',
]

knob_list = [
    "save0_0",
    "save0_1",
    "save1_0",
    "save1_1",
    "save2_0",
    "save2_1",

    "rdbcompression",
    "rdbchecksum",
    "rdb-save-incremental-fsync",

    "activedefrag",
    "active-defrag-threshold-lower",
    "active-defrag-threshold-upper",
    "active-defrag-cycle-min",
    "active-defrag-cycle-max",
    
    "maxmemory",
    
    "maxmemory-policy",
    "maxmemory-samples",
    "lazyfree-lazy-eviction",
    "lazyfree-lazy-expire",
    "lazyfree-lazy-server-del",

    "hash-max-ziplist-entries", 
    "hash-max-ziplist-value",
    "activerehashing",
    "hz",
    "dynamic-hz"
]

activedefrag_knobs = [
    "activedefrag",
    "active-defrag-threshold-lower",
    "active-defrag-threshold-upper",
    "active-defrag-cycle-min",
    "active-defrag-cycle-max",
]

In [59]:
# 데이터 불러오기 internal metrics와 knob들의 상관관계분석
# internal_csv = pd.read_csv('./data/internal_metrcis0.csv')

file_dir = '../data/'
file_name = 'internal_metrics_r10000_rdb2.csv'

internal_csv = pd.read_csv(file_dir+file_name)

# 1250 개 추출
internal_csv = internal_csv.iloc[0:1250,:]
internal_csv.shape

(1250, 124)

In [60]:
# 데이터 불러오기 knob_list
# knobs = pd.read_csv('./data/result_config0.csv')

file_dir = '../data/'
file_name = 'result_config_r10000_rdb.csv'

knobs = pd.read_csv(file_dir + file_name)
# 1250 개 추출
knobs = knobs.iloc[0:1250,:]
knobs.shape

(1250, 32)

In [6]:
# 컬럼 추출 함수
def extract_data(cluster_list, df):
    extracted_df = pd.DataFrame()
    for elem in cluster_list:
        extracted_df[elem] = df[elem]

    return extracted_df    

## 데이터 전처리

In [7]:
# 클러스터 합치기
cluster = cluster0+cluster1

In [8]:
# 클러스터 요소에 해당되는 metrics 25개 추출
total_internal_df = extract_data(cluster, internal_csv)

In [9]:
# throughput 결측치 제거
total_internal_df.dropna()

,uptime_in_seconds,hz,lru_clock,used_memory_peak,used_memory_overhead,allocator_allocated,allocator_active,allocator_resident,maxmemory,maxmemory_human,...,instantaneous_input_kbps,instantaneous_output_kbps,used_cpu_sys,used_cpu_user,used_memory_rss,used_memory_rss_human,rss_overhead_ratio,rss_overhead_bytes,mem_fragmentation_ratio,mem_fragmentation_bytes
0,6,23,4429256,1826632,1062680,1781896,2031616,5050368,2097152000,1.95,...,1063.66,70.91,0.224627,0.106211,4722688,4.50,0.94,-327680,2.71,2978048
1,6,36,4429275,1826632,1062680,1781864,2031616,4980736,1258291200,1.17,...,1130.28,75.35,0.182061,0.100586,4722688,4.50,0.95,-258048,2.71,2978048
2,6,37,4429287,1826632,1062680,1781736,2031616,4956160,1887436800,1.76,...,1128.91,75.26,0.199448,0.080606,4698112,4.48,0.95,-258048,2.69,2953472
3,6,16,4429299,1826632,1062680,1781896,2031616,4968448,2621440000,2.44,...,1476.66,98.44,0.194226,0.087181,4702208,4.48,0.95,-266240,2.70,2957568
4,6,21,4429312,1826632,1062680,1781896,2031616,5042176,2202009600,2.05,...,971.93,64.79,0.214280,0.116201,4710400,4.49,0.93,-331776,2.70,2965760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,7,8,4444376,1826632,1062680,1781896,2031616,5042176,2621440000,2.44,...,732.42,48.83,0.198279,0.084415,4726784,4.51,0.94,-315392,2.71,2982144
1246,6,37,4444389,1826632,1062680,1781704,2031616,4980736,1258291200,1.17,...,1130.28,75.35,0.165889,0.105376,4698112,4.48,0.94,-282624,2.69,2953472
1247,6,10,4444401,1826632,1062680,1781704,2031616,5042176,1992294400,1.86,...,915.53,61.04,0.176508,0.087738,4722688,4.50,0.94,-319488,2.71,2978048
1248,6,38,4444413,1826632,1062680,1782280,2031616,4980736,1468006400,1.37,...,1171.38,78.09,0.207251,0.092774,4718592,4.50,0.95,-262144,2.70,2973952


In [10]:
# internal_csv 결측치 확인 -> 없음!
total_internal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1250 entries, 0 to 1249
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   uptime_in_seconds          1250 non-null   int64  
 1   hz                         1250 non-null   int64  
 2   lru_clock                  1250 non-null   int64  
 3   used_memory_peak           1250 non-null   int64  
 4   used_memory_overhead       1250 non-null   int64  
 5   allocator_allocated        1250 non-null   int64  
 6   allocator_active           1250 non-null   int64  
 7   allocator_resident         1250 non-null   int64  
 8   maxmemory                  1250 non-null   int64  
 9   maxmemory_human            1250 non-null   float64
 10  allocator_frag_bytes       1250 non-null   int64  
 11  allocator_rss_ratio        1250 non-null   float64
 12  allocator_rss_bytes        1250 non-null   int64  
 13  rdb_last_save_time         1250 non-null   int64

In [11]:
# maxmemory 처리 throughtput은 처리할 필요없다.
total_internal_df['maxmemory']

0       2097152000
1       1258291200
2       1887436800
3       2621440000
4       2202009600
           ...    
1245    2621440000
1246    1258291200
1247    1992294400
1248    1468006400
1249    1572864000
Name: maxmemory, Length: 1250, dtype: int64

In [12]:
# maxmemory 처리 knob
knobs['maxmemory'] = knobs['maxmemory'].str.strip('mb')  # 단위 자르기
knobs['maxmemory'] = pd.to_numeric(knobs['maxmemory'])  # 숫자형으로 바꾸기

### activedefrag 처리
* subTotal_knobs_Active : 약 1300개 이며 activedefrag가 있는 경우 
* subTotal_knobs_NoActive: 약 1300개 이며 activedefrag가 nan값으로 제거한 경우
* subTotal_knobs_T : 2500개 이며 activedefrag 상관없이 전체 데이터


* total_internal_Active : 약 1300개 이며 activedefrag가 있는 경우 
* total_internal_NoActive: 약 1300개 이며 activedefrag가 nan값으로 제거한 경우
* total_internal_T : 2500개 이며 activedefrag 상관없이 전체 데이터

In [13]:
# knob_list 만 추출, AOF 관련 제외
subTotal_knobs = extract_data(knob_list, knobs)

In [14]:
subTotal_knobs

,save0_0,save0_1,save1_0,save1_1,save2_0,save2_1,rdbcompression,rdbchecksum,rdb-save-incremental-fsync,activedefrag,...,maxmemory-policy,maxmemory-samples,lazyfree-lazy-eviction,lazyfree-lazy-expire,lazyfree-lazy-server-del,hash-max-ziplist-entries,hash-max-ziplist-value,activerehashing,hz,dynamic-hz
0,1056,4,114,13,57,8497,yes,no,no,NaN,...,volatile-lru,6,yes,no,no,615,232,no,23,no
1,805,2,343,23,47,8561,no,yes,no,NaN,...,allkeys-lru,7,yes,no,no,496,156,yes,36,no
2,776,6,194,46,61,11337,no,yes,yes,yes,...,volatile-random,5,no,yes,no,494,135,no,37,no
3,742,9,295,59,57,8989,yes,yes,yes,NaN,...,allkeys-lru,7,no,no,no,491,76,yes,16,no
4,985,5,225,90,41,11978,no,no,no,NaN,...,volatile-lru,5,yes,no,yes,332,80,yes,21,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,851,5,121,84,79,11338,yes,no,no,NaN,...,noeviction,3,no,yes,no,623,244,no,8,no
1246,825,8,443,19,49,10608,yes,no,yes,yes,...,noeviction,7,no,yes,yes,363,34,no,37,no
1247,908,7,255,88,64,8293,no,no,yes,yes,...,volatile-ttl,7,no,no,yes,572,229,yes,10,no
1248,1089,5,179,81,40,8054,no,yes,yes,yes,...,allkeys-lfu,6,yes,yes,no,375,32,no,38,yes


In [15]:
#activedefrag knobs 분리

# activedefrag O 존재
subTotal_knobs_Active = subTotal_knobs.dropna()
Active_index = list(subTotal_knobs_Active.index)  # 인덱스

# activedefrag X 존재하지 않는 나머지 반
subTotal_knobs_NoActive = subTotal_knobs.loc[subTotal_knobs['activedefrag'].isna()].dropna(axis=1)
NoActive_index = list(subTotal_knobs_NoActive.index)  # 인덱스

# activedefrag O and X 전체 
subTotal_knobs_T = subTotal_knobs.dropna(axis=1)
total_index = list(subTotal_knobs_T.index)   # 인덱스

In [16]:
# knobs에서 타입별로 분리 (수치, 불리언, 불리언아닌 문자)

def varType(subTotal_knobs):
    object_var = []
    numerical_var = []  # 문자, 숫자 타입
    
    for idx in subTotal_knobs.dtypes.index:
        if subTotal_knobs.dtypes[idx] == 'O':  # 문자일 경우
            object_var.append(idx)
        else: 
            numerical_var.append(idx)  # 숫자일 경우

    boolean_var = []
    booleanNo_var = []  # 2개, 2개이상

    for col in object_var:
        col_count = len(subTotal_knobs[col].unique())

        if col_count == 2:  # 2개일 경우
            boolean_var.append(col)
        elif col_count > 2:  # 2개 이상일 경우
            booleanNo_var.append(col)
    
    return numerical_var, boolean_var, booleanNo_var

# 수치, 불리언, 불리언아닌것 (전체)
numerical_var, boolean_var, NobooleanNo_var = varType(subTotal_knobs_T)

# 수치, 불리언, 불리언아닌것 (activedefrag)
numericalActive_var, booleanActive_var, NobooleanNoActive_var = varType(subTotal_knobs_Active)

# 수치, 불리언, 불리언아닌것 (activedefrag 없는것)
numericalNoActive_var, booleanNoActive_var, NobooleanNoActive_var = varType(subTotal_knobs_NoActive)


In [17]:
# activedefrag O
total_internal_Active = total_internal_df.iloc[Active_index]

# activedefrag X
total_internal_NoActive = total_internal_df.iloc[NoActive_index]

# activedefrag O and X
total_internal_T = total_internal_df.iloc[total_index]

---

## 상관분석 함수

In [18]:
def get_outlier(df=None, column=None, weight=1.5):
    
    # target 값과 상관관계가 높은 열을 우선적으로 진행
    quantile_25 = np.percentile(df[column].values, 25)
    quantile_75 = np.percentile(df[column].values, 75)

    IQR = quantile_75 - quantile_25
    IQR_weight = IQR*weight

    lowest = quantile_25 - IQR_weight
    highest = quantile_75 + IQR_weight

    outlier_idx = df[column][ (df[column] < lowest) | (df[column] > highest) ].index
    return outlier_idx

In [19]:
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import kendalltau


# knob하나 internal 여러개 상관계수 구하기

# pearson
## 모수 검정
def cal_pearsonr(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = pearsonr(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df

# kendall
## 변수값 대신 순위로 바꿔서 이용, 비모수 검정
## 샘플사이저 적거나, 데이터 동률이 많을 때 유용
def cal_kendalltau(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = kendalltau(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df

# 스피어만
## 변수값 대신 순위로 바꿔서 이용, 비모수 검정
## 데이터 내 편차와 에러에 만감하며, 일반적으로 켄달보다 높은 값을 가진다.
def cal_spearmanr(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = spearmanr(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df


# knob하나 internal 여러개 상관계수 구하기 이상치 제거 버전
# pearson
## 모수 검정
def cal_pearsonr_outlier(knobs, internal_df):

    for internal in internal_df:
        outlier_idx = get_outlier(df=internal_df, column=internal, weight=1.5)  # 이상치 인덱스
        
        internal_df_no_outlier = internal_df[internal].drop(outlier_idx, axis=0)
        knobs_no_outlier = knobs.drop(outlier_idx, axis=0)

        result = pearsonr(knobs_no_outlier, internal_df_no_outlier)
        if result[1] <= 0.05:
            print(internal)
            print(result[0])

# kendall
## 변수값 대신 순위로 바꿔서 이용, 비모수 검정
## 샘플사이저 적거나, 데이터 동률이 많을 때 유용
def cal_kendalltau_outlier(knobs, internal_df):

    for internal in internal_df:
        outlier_idx = get_outlier(df=internal_df, column=internal, weight=1.5)  # 이상치 인덱스
        
        internal_df_no_outlier = internal_df[internal].drop(outlier_idx, axis=0)
        knobs_no_outlier = knobs.drop(outlier_idx, axis=0)

        result = kendalltau(knobs_no_outlier, internal_df_no_outlier)
        if result[1] <= 0.05:
            print(internal)
            print(result[0])

    
# 스피어만
## 변수값 대신 순위로 바꿔서 이용, 비모수 검정
## 데이터 내 편차와 에러에 만감하며, 일반적으로 켄달보다 높은 값을 가진다.
def cal_spearmanr_outlier(knobs, internal_df):

    for internal in internal_df:
        outlier_idx = get_outlier(df=internal_df, column=internal, weight=1.5)  # 이상치 인덱스
        
        internal_df_no_outlier = internal_df[internal].drop(outlier_idx, axis=0)
        knobs_no_outlier = knobs.drop(outlier_idx, axis=0)

        result = spearmanr(knobs_no_outlier, internal_df_no_outlier)
        if result[1] <= 0.05:
            print(internal)
            print(result[0])



In [20]:
# knob 하나별로 3가지 상관분석 수행
def extract_pvalue(knobs, internal_df):
    print('pearson')
    df = cal_pearsonr(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()
    
    print('kendalltau')
    df = cal_kendalltau(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()
    
    print('spearman')
    df = cal_spearmanr(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()

def extract_pvalue_outlier(knobs, internal_df):
    print('pearson')
    cal_pearsonr_outlier(knobs, internal_df)
    print()
    
    print('kendalltau')
    cal_kendalltau_outlier(knobs, internal_df)
    print()
    
    print('spearman')
    cal_spearmanr_outlier(knobs, internal_df)
    print()
    

# knob 리스트 각각 별로 extract_pvalue 수행
def extract_pvalue_knob(knob_list, subTotal_knobs, total_internal):
    for knob in knob_list:
        print('###{}###'.format(knob))
        extract_pvalue(subTotal_knobs[knob], total_internal)
        
        
# knob 리스트 각각 별로 extract_pvalue 수행 이상치 제거 
def extract_pvalue_knob_outlier(knob_list, subTotal_knobs, total_internal):
    for knob in knob_list:
        print('###{}###'.format(knob))
        extract_pvalue_outlier(subTotal_knobs[knob], total_internal)

---

## activedefrag 상관계수
* subTotal_knobs_Active
* total_internal_Active

### numericalActive_var :
* active-defrag-threshold-lower 
* active-defrag-threshold-upper 
* active-defrag-cycle-min
* active-defrag-cycle-max

In [21]:
numericalActive_var
numericalActive_var_new = numericalActive_var[6:10]
numericalActive_var_new

['active-defrag-threshold-lower',
 'active-defrag-threshold-upper',
 'active-defrag-cycle-min',
 'active-defrag-cycle-max']

In [22]:
extract_pvalue_knob(numericalActive_var_new, subTotal_knobs_Active,total_internal_Active)

###active-defrag-threshold-lower###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###active-defrag-threshold-upper###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###active-defrag-cycle-min###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###active-defrag-cycle-max###
pearson
                         corr   p-value
allocator_allocated  0.079653  0.047604
allocator_active     0.081811  0.041878
used_cpu_sys         0.087494  0.029510

kendalltau
                      corr   p-value
allocator_active  0.066058  0.048969
used_cpu_sys      0.064440  0.019124

spearman
               

In [23]:
# cluster 별 knob들의 상관계수 합  
active_result = pd.DataFrame(index = numericalActive_var_new)

cluster_list = [cluster0, cluster1]

for idx, cluster in enumerate(cluster_list):
    cluster_val = []
    for knob in numericalActive_var_new:
        df = cal_pearsonr(subTotal_knobs_Active[knob], total_internal_Active.loc[:,cluster])
        cluster_val.append(df['corr'].abs().mean())

    active_result['cluster'+str(idx)] = cluster_val

In [24]:
active_result

,cluster0,cluster1
active-defrag-threshold-lower,0.036746,0.021077
active-defrag-threshold-upper,0.025162,0.022983
active-defrag-cycle-min,0.020677,0.016800
active-defrag-cycle-max,0.037489,0.016677


In [25]:
# knob이 어떤 클러스터에 속하는지 확인 가능 함수
def get_cluster(df):
    cluster_list = df.columns.tolist()  # [cluster0, cluster1]
    for knob_index in list(df.index):
        max_index = np.argmax(df.loc[knob_index,:].values)
        print(knob_index+' : ', end='')
        print(cluster_list[max_index])    

In [27]:
get_cluster(active_result)

active-defrag-threshold-lower : cluster0
active-defrag-threshold-upper : cluster0
active-defrag-cycle-min : cluster0
active-defrag-cycle-max : cluster0


### 이상치 제거 후 상관계수

In [53]:
extract_pvalue_knob_outlier(numericalActive_var_new, subTotal_knobs_Active,total_internal_Active)

###active-defrag-threshold-lower###
pearson

kendalltau

spearman

###active-defrag-threshold-upper###
pearson

kendalltau

spearman

###active-defrag-cycle-min###
pearson

kendalltau

spearman

###active-defrag-cycle-max###
pearson

kendalltau

spearman



---

## 전체 knob
* subTotal_knobs_T
* total_internal_T

### numerical_var

In [28]:
numerical_var

['save0_0',
 'save0_1',
 'save1_0',
 'save1_1',
 'save2_0',
 'save2_1',
 'maxmemory',
 'maxmemory-samples',
 'hash-max-ziplist-entries',
 'hash-max-ziplist-value',
 'hz']

In [29]:
extract_pvalue_knob(numerical_var, subTotal_knobs_T,total_internal_T)

###save0_0###
pearson
                             corr   p-value
used_memory_rss         -0.060827  0.031525
used_memory_rss_human   -0.061065  0.030865
mem_fragmentation_bytes -0.057591  0.041769

kendalltau
                          corr   p-value
allocator_frag_bytes -0.044611  0.025869

spearman
                          corr   p-value
allocator_frag_bytes -0.062424  0.027317

###save0_1###
pearson
        corr  p-value
hz -0.060676  0.03195

kendalltau
                     corr   p-value
hz              -0.042579  0.034533
maxmemory_human -0.040261  0.048328

spearman
                     corr   p-value
hz              -0.059611  0.035089
maxmemory_human -0.056976  0.044006

###save1_0###
pearson
                         corr   p-value
allocator_allocated -0.061530  0.029609
allocator_active    -0.063341  0.025126

kendalltau
                      corr   p-value
allocator_active -0.049544  0.031754

spearman
                      corr  p-value
allocator_active -0.060743  0.03176


In [30]:
subTotal_knobs_T  # knobs에서 aof 관련 제거한 것
total_internal_T  # 전체

,uptime_in_seconds,hz,lru_clock,used_memory_peak,used_memory_overhead,allocator_allocated,allocator_active,allocator_resident,maxmemory,maxmemory_human,...,instantaneous_input_kbps,instantaneous_output_kbps,used_cpu_sys,used_cpu_user,used_memory_rss,used_memory_rss_human,rss_overhead_ratio,rss_overhead_bytes,mem_fragmentation_ratio,mem_fragmentation_bytes
0,6,23,4429256,1826632,1062680,1781896,2031616,5050368,2097152000,1.95,...,1063.66,70.91,0.224627,0.106211,4722688,4.50,0.94,-327680,2.71,2978048
1,6,36,4429275,1826632,1062680,1781864,2031616,4980736,1258291200,1.17,...,1130.28,75.35,0.182061,0.100586,4722688,4.50,0.95,-258048,2.71,2978048
2,6,37,4429287,1826632,1062680,1781736,2031616,4956160,1887436800,1.76,...,1128.91,75.26,0.199448,0.080606,4698112,4.48,0.95,-258048,2.69,2953472
3,6,16,4429299,1826632,1062680,1781896,2031616,4968448,2621440000,2.44,...,1476.66,98.44,0.194226,0.087181,4702208,4.48,0.95,-266240,2.70,2957568
4,6,21,4429312,1826632,1062680,1781896,2031616,5042176,2202009600,2.05,...,971.93,64.79,0.214280,0.116201,4710400,4.49,0.93,-331776,2.70,2965760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,7,8,4444376,1826632,1062680,1781896,2031616,5042176,2621440000,2.44,...,732.42,48.83,0.198279,0.084415,4726784,4.51,0.94,-315392,2.71,2982144
1246,6,37,4444389,1826632,1062680,1781704,2031616,4980736,1258291200,1.17,...,1130.28,75.35,0.165889,0.105376,4698112,4.48,0.94,-282624,2.69,2953472
1247,6,10,4444401,1826632,1062680,1781704,2031616,5042176,1992294400,1.86,...,915.53,61.04,0.176508,0.087738,4722688,4.50,0.94,-319488,2.71,2978048
1248,6,38,4444413,1826632,1062680,1782280,2031616,4980736,1468006400,1.37,...,1171.38,78.09,0.207251,0.092774,4718592,4.50,0.95,-262144,2.70,2973952


In [31]:
# cluster 별 knob들의 상관계수 합  
numerical_result = pd.DataFrame(index = numerical_var)

cluster_list = [cluster0, cluster1]

for idx, cluster in enumerate(cluster_list):
    cluster_val = []
    for knob in numerical_var:
        df = cal_spearmanr(subTotal_knobs_T[knob], total_internal_T.loc[:,cluster])
        cluster_val.append(df['corr'].abs().mean())

    numerical_result['cluster'+str(idx)] = cluster_val

In [32]:
numerical_result

,cluster0,cluster1
save0_0,0.017283,0.017196
save0_1,0.028183,0.019866
save1_0,0.023317,0.019728
save1_1,0.015805,0.014818
save2_0,0.021794,0.018200
save2_1,0.027201,0.007894
maxmemory,0.119283,0.018823
maxmemory-samples,0.021128,0.019050
hash-max-ziplist-entries,0.028326,0.026155
hash-max-ziplist-value,0.023843,0.030662


In [33]:
get_cluster(numerical_result)

save0_0 : cluster0
save0_1 : cluster0
save1_0 : cluster0
save1_1 : cluster0
save2_0 : cluster0
save2_1 : cluster0
maxmemory : cluster0
maxmemory-samples : cluster0
hash-max-ziplist-entries : cluster0
hash-max-ziplist-value : cluster1
hz : cluster0


In [34]:
numerical_result

,cluster0,cluster1
save0_0,0.017283,0.017196
save0_1,0.028183,0.019866
save1_0,0.023317,0.019728
save1_1,0.015805,0.014818
save2_0,0.021794,0.018200
save2_1,0.027201,0.007894
maxmemory,0.119283,0.018823
maxmemory-samples,0.021128,0.019050
hash-max-ziplist-entries,0.028326,0.026155
hash-max-ziplist-value,0.023843,0.030662


In [53]:
# extract_pvalue_knob_outlier(numerical_var, subTotal_knobs_T,total_internal_T)

---

## Activedefrag 없는 knob들
* subTotal_knobs_Active
* total_internal_Active
active인거는 신경 안써도 된다.
### numericalActive_var

In [35]:
numericalActive_var

['save0_0',
 'save0_1',
 'save1_0',
 'save1_1',
 'save2_0',
 'save2_1',
 'active-defrag-threshold-lower',
 'active-defrag-threshold-upper',
 'active-defrag-cycle-min',
 'active-defrag-cycle-max',
 'maxmemory',
 'maxmemory-samples',
 'hash-max-ziplist-entries',
 'hash-max-ziplist-value',
 'hz']

In [36]:
extract_pvalue_knob(numericalActive_var, subTotal_knobs_Active,total_internal_Active)

###save0_0###
pearson
                       corr   p-value
lru_clock           0.09352  0.019957
rdb_last_save_time  0.09352  0.019957

kendalltau
                          corr   p-value
lru_clock             0.062364  0.020436
allocator_frag_bytes -0.067921  0.023071
rdb_last_save_time    0.062364  0.020436

spearman
                          corr   p-value
lru_clock             0.093162  0.020438
allocator_frag_bytes -0.090087  0.025002
rdb_last_save_time    0.093162  0.020438

###save0_1###
pearson
        corr   p-value
hz -0.096756  0.016038

kendalltau
        corr   p-value
hz -0.068438  0.016919

spearman
        corr   p-value
hz -0.094851  0.018254

###save1_0###
pearson
                             corr   p-value
allocator_allocated     -0.090421  0.024467
allocator_active        -0.092773  0.020973
used_memory_rss          0.085178  0.034110
used_memory_rss_human    0.084807  0.034901
mem_fragmentation_ratio  0.097115  0.015648
mem_fragmentation_bytes  0.090401  0.024499


## other knob
* subTotal_knobs_NoActive
* total_internal_NoActive
active인거는 신경 안써도 된다.
### numericalNoActive_var

In [37]:
numericalNoActive_var

['save0_0',
 'save0_1',
 'save1_0',
 'save1_1',
 'save2_0',
 'save2_1',
 'maxmemory',
 'maxmemory-samples',
 'hash-max-ziplist-entries',
 'hash-max-ziplist-value',
 'hz']

In [38]:
extract_pvalue_knob(numericalNoActive_var, subTotal_knobs_NoActive,total_internal_NoActive)

###save0_0###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
                           corr   p-value
allocator_resident    -0.058122  0.037439
allocator_rss_ratio   -0.057116  0.045146
used_memory_rss_human -0.059138  0.042209

spearman
                           corr   p-value
allocator_resident    -0.083995  0.034903
allocator_rss_ratio   -0.081636  0.040361
used_memory_rss_human -0.081083  0.041741

###save0_1###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###save1_0###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
Empty DataFrame
Columns: [corr, p-value]
Index: []

spearman
Empty DataFrame
Columns: [corr, p-value]
Index: []

###save1_1###
pearson
Empty DataFrame
Columns: [corr, p-value]
Index: []

kendalltau
                      corr   p-value
used_memory_peak -0.065432  0.045441

spearman
   

---

## pointbiserialr 분석

In [39]:
from scipy.stats import pointbiserialr
# pointbiserialr
## yes,no 데이터와 연속형 데이터 상관분석
def cal_pointbiserialr(knobs, internal_df):
    df = pd.DataFrame(columns = ['corr', 'p-value'])
    for internal in internal_df:
        result = pointbiserialr(knobs, internal_df[internal])
        df.loc[internal] = [result[0], result[1]]
    return df

# knob 하나별로 3가지 상관분석 수행
def extract_pvalue(knobs, internal_df):
    print('pointbierialr')
    df = cal_pointbiserialr(knobs, internal_df)
    is_pvalue = df['p-value'] <= 0.05
    print(df[is_pvalue])
    print()

    
# knob 리스트 각각 별로 extract_pvalue 수행
def extract_pvalue_knob_point(knob_list, subTotal_knobs, total_internal):
    for knob in knob_list:
        print('###{}###'.format(knob))
        extract_pvalue(subTotal_knobs[knob], total_internal)


# 이상치 제거 버전
def cal_pointbiserialr_outlier(knobs, internal_df):

    for internal in internal_df:
        outlier_idx = get_outlier(df=internal_df, column=internal, weight=1.5)  # 이상치 인덱스
        
        internal_df_no_outlier = internal_df[internal].drop(outlier_idx, axis=0)
        knobs_no_outlier = knobs.drop(outlier_idx, axis=0)

        result = pointbiserialr(knobs_no_outlier, internal_df_no_outlier)
        if result[1] <= 0.05:
            print(internal)
            print(result[0])

# knob 하나별로 3가지 상관분석 수행
def extract_pvalue_outlier(knobs, internal_df):
    print('pointbierialr')
    cal_pointbiserialr(knobs, internal_df)
    print()

# knob 리스트 각각 별로 extract_pvalue 수행
def extract_pvalue_knob_point_outlier(knob_list, subTotal_knobs, total_internal):
    for knob in knob_list:
        print('###{}###'.format(knob))
        extract_pvalue(subTotal_knobs[knob], total_internal)


## other knob(activedefrag 제외)
* subTotal_knobs_T
* total_internal_T

### boolean_var

In [46]:
boolean_var

['rdbcompression',
 'rdbchecksum',
 'rdb-save-incremental-fsync',
 'lazyfree-lazy-eviction',
 'lazyfree-lazy-expire',
 'lazyfree-lazy-server-del',
 'activerehashing',
 'dynamic-hz']

In [47]:
#  yes, no -> 1, 0으로 변경
subTotal_knobs_T_bool = subTotal_knobs_T[boolean_var]  ## boolean 변수 df 추출
subTotal_knobs_T_point = subTotal_knobs_T_bool.replace(['yes', 'no'], [1,0])  ## 1, 0 변경

In [48]:
extract_pvalue_knob_point(boolean_var, subTotal_knobs_T_point, total_internal_T)

###rdbcompression###
pointbierialr
                        corr   p-value
rss_overhead_ratio  0.061593  0.029441
rss_overhead_bytes  0.058881  0.037391

###rdbchecksum###
pointbierialr
Empty DataFrame
Columns: [corr, p-value]
Index: []

###rdb-save-incremental-fsync###
pointbierialr
                      corr   p-value
used_memory_peak -0.056908  0.044259
used_cpu_user    -0.058670  0.038077

###lazyfree-lazy-eviction###
pointbierialr
Empty DataFrame
Columns: [corr, p-value]
Index: []

###lazyfree-lazy-expire###
pointbierialr
                         corr   p-value
allocator_resident   0.059353  0.035888
allocator_rss_ratio  0.056251  0.046771
allocator_rss_bytes  0.058301  0.039308
rss_overhead_ratio  -0.060744  0.031756
rss_overhead_bytes  -0.061312  0.030192

###lazyfree-lazy-server-del###
pointbierialr
                        corr   p-value
uptime_in_seconds  -0.080162  0.004570
lru_clock          -0.091762  0.001163
rdb_last_save_time -0.091761  0.001163

###activerehashing###
poi

In [49]:
# cluster 별 knob들의 상관계수 합
bool_result = pd.DataFrame(index = boolean_var)

cluster_list = [cluster0, cluster1]

for idx, cluster in enumerate(cluster_list):
    cluster_val = []
    for knob in boolean_var:
        df = cal_pointbiserialr(subTotal_knobs_T_point[knob], total_internal_T.loc[:,cluster])
        cluster_val.append(df['corr'].abs().mean())

    bool_result['cluster'+str(idx)] = cluster_val

In [50]:
bool_result

,cluster0,cluster1
rdbcompression,0.024836,0.047281
rdbchecksum,0.018976,0.003934
rdb-save-incremental-fsync,0.025766,0.004363
lazyfree-lazy-eviction,0.016096,0.022092
lazyfree-lazy-expire,0.026333,0.033035
lazyfree-lazy-server-del,0.032339,0.006767
activerehashing,0.030126,0.009637
dynamic-hz,0.024643,0.026360


In [51]:
get_cluster(bool_result)

rdbcompression : cluster1
rdbchecksum : cluster0
rdb-save-incremental-fsync : cluster0
lazyfree-lazy-eviction : cluster1
lazyfree-lazy-expire : cluster1
lazyfree-lazy-server-del : cluster0
activerehashing : cluster0
dynamic-hz : cluster1


In [57]:
#이상치 제거
extract_pvalue_knob_point_outlier(boolean_var, subTotal_knobs_T_point, total_internal_T)

###rdbcompression###
pointbierialr
                            corr   p-value
total_net_output_bytes -0.056501  0.004715

###rdbchecksum###
pointbierialr
Empty DataFrame
Columns: [corr, p-value]
Index: []

###rdb-save-incremental-fsync###
pointbierialr
Empty DataFrame
Columns: [corr, p-value]
Index: []

###lazyfree-lazy-eviction###
pointbierialr
                         corr   p-value
rss_overhead_ratio  -0.046519  0.020017
rss_overhead_bytes  -0.048922  0.014431
allocator_resident   0.043003  0.031551
allocator_rss_ratio  0.043123  0.031080
allocator_rss_bytes  0.043044  0.031389

###lazyfree-lazy-expire###
pointbierialr
Empty DataFrame
Columns: [corr, p-value]
Index: []

###lazyfree-lazy-server-del###
pointbierialr
        corr   p-value
hz -0.045644  0.022474

###activerehashing###
pointbierialr
                         corr   p-value
allocator_allocated -0.044451  0.026248
allocator_active    -0.040469  0.043047

###dynamic-hz###
pointbierialr
                     corr   p-value
ma

---

In [ ]:
extract_pvalue_knob(numerical_var, subTotal_knobs_T,total_internal_T)
extract_pvalue_knob(numericalActive_var_new, subTotal_knobs_Active,total_internal_Active)
extract_pvalue_knob_point(boolean_var, subTotal_knobs_T_point, total_internal_T)